## **Model Tuning: Transfer Learning with Resnet**

In [2]:
# import necessary libraries
import os
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import tensorflow.keras.models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import ResNet50

## **Load image dataframe**

In [3]:
# choose day or night dataframe for reference
infile = 'day_df.pkl'
#infile = 'night_df.pkl'
path = '/home/ubuntu/michael/my_pickles/'

In [4]:
# set reference dataframe
sample_df = pd.read_pickle(path + infile)

## **Set source directories**

In [5]:
# to access day images only
train_dir = '/home/ubuntu/michael/day/train'
val_dir = '/home/ubuntu/michael/day/validate'

In [6]:
# to access night images only
#train_dir = '/home/ubuntu/michael/night/train'
#val_dir = '/home/ubuntu/michael/night/validate'

## **Loading images with Keras**

In [7]:
# clear memory
K.clear_session()

In [8]:
# set image size and shape
img_size = (224, 224)
img_shape = (224, 224, 3)

## **Load images from directories**

In [9]:
# create data generators
datagen = ImageDataGenerator(rescale = 1.0 / 255.0) #scale pixel values to be in the range of 0-1

In [10]:
# include 'out-of-the-box' image augmentation
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, 
                                   width_shift_range = 0.2, height_shift_range = 0.2,
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

In [11]:
# prepare training iterator
train_it = datagen.flow_from_directory(directory = train_dir, classes = ['nofog', 'fog'], class_mode = 'binary', 
                    batch_size = 64, target_size = img_size) 
# flip classes due to alphabetical order; fog would be 0 since it preceeds nofog in the alphabet

Found 897 images belonging to 2 classes.


In [12]:
# assign class names and indices
class_names = train_it.classes # lists all assignments of images [0 = nofog; 1 = fog]
class_indices = train_it.class_indices

In [13]:
# prepare validation iterator
valid_it = datagen.flow_from_directory(directory = val_dir, classes = ['nofog', 'fog'], class_mode = 'binary', 
                    batch_size = 16, target_size = img_size)
# flip classes due to alphabetical order; fog would be 0 since it preceeds nofog in the alphabet

Found 112 images belonging to 2 classes.


In [14]:
class_names = valid_it.classes
class_indices = valid_it.class_indices

## **Load Resent model as convoutional base**

In [15]:
# load pretrained base model Resnet
conv_base = ResNet50(input_shape = img_shape, include_top = False, weights = 'imagenet')

In [16]:
# freeze base layers
conv_base.trainable = False

## **Add classifier on top of convolutional base**

In [17]:
# create classification model
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))

In [32]:
# view classification architecture
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              102761472 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1025      
Total params: 126,350,209
Trainable params: 102,762,497
Non-trainable params: 23,587,712
_________________________________________________________________


In [18]:
# compile model
model.compile(loss = 'binary_crossentropy', 
              optimizer =  RMSprop(lr = 1e-4), 
              metrics = ['acc'])

In [19]:
# set early stopping
es = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, 
                                      patience=15, restore_best_weights = True)

In [20]:
# choose day or night output folder
outputFolder = '/home/ubuntu/michael/model_output/Resnet/'
#outputFolder = '/home/ubuntu/michael/model_output/Resnet/day/'
#outputFolder = '/home/ubuntu/michael/model_output/Resnet/night/'
if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)

In [21]:
# define chekpoint path and add checkpoints
checkpoint_path = outputFolder + 'Resnet_model.hdf5'

# create a model checkpoint when condtions are met
cp = ModelCheckpoint(checkpoint_path, monitor = 'val_acc', mode = 'max', save_best_only=True)

In [22]:
# fit model with train itertor and use valid_it as a validation dataset during training
history = model.fit(train_it, steps_per_epoch = len(train_it), 
                    validation_data = valid_it, 
                    validation_steps = len(valid_it), 
                    epochs = 300, verbose = 1, callbacks=[es, cp]) 

In [23]:
# evaluate model
score = model.evaluate(valid_it, steps=len(valid_it), verbose = 1)

In [25]:
# save model validation score
with open(outputFolder + 'Resnet_score.pkl', 'wb') as f:pickle.dump(score, f)

In [44]:
# access training and loss values from history object
loss_values = history.history['loss']
acc_values = history.history['acc']
valLoss_values = history.history['val_loss']
valAccuracy_values = history.history['val_acc']

# create a dataframe to store training accuracy and loss values
history_df = pd.DataFrame()
history_df['Training Loss'] = history.history['loss']
history_df['Training Accuracy'] = history.history['acc']
history_df['Validation Loss'] = history.history['val_loss']
history_df['Validation Accuracy'] = history.history['val_acc']

In [26]:
# save accuracy and loss values to a dataframe
history_df.to_pickle(outputFolder + 'Resnet_history.pkl') #/home/ubuntu/michael/my_historys